In [13]:
import cv2
import numpy as np
import scipy.io
import math
from skimage import data
from skimage import io
from skimage import color
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from skimage import filters
from scipy import signal
from skimage.filters.rank import gradient
from scipy.ndimage import gaussian_filter
from numpy import linalg as LA
from skimage.transform import pyramid_gaussian
from skimage.morphology import disk
from collections import OrderedDict
from skimage import transform as tf
from scipy.spatial import distance
from random import randint
import glob
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
import dlib
import face_recognition
from matplotlib.pyplot import imshow, show

In [14]:
# This function takes as input an image, the right point of the face rPt, and the left point of the face lPt.
# I add 20 pixels around the face and crop and return the image.
def crop_img(img,rPt,lPt):
    
    x1=rPt[1]+20
    y1=rPt[0]-20
    x2=lPt[1]-20
    y2=lPt[0]+20

    crop_img1 = img[y1:y2, x2:x1]

    return crop_img1

In [15]:
# This function takes a frame of a video as input.
# I use the facefeatures list to add all the face features of the frame to it.
# I get the locations of the faces using the face_recognition.face_locations().
# I loop over the faces and crop them using the crop function.
# Then, I get the face features from the cropped image.
# If face features were found, I calculate the left eye, right eye, nose and mouth and
# add them to the features list.

def get_feat(frame):

    facefeatures = []
        
    img = face_recognition.load_image_file(frame)
    face_location = face_recognition.face_locations(img)

    for face in face_location:
        features = []

        y1 = face[0]
        x1 = face[1]
        y2 = face[2]
        x2 = face[3]

        rPt = [y1,x1]
        lPt = [y2,x2]

        cropped = crop_img(img,rPt,lPt)

        faceFeat = face_recognition.face_landmarks(cropped)

        if (faceFeat):
            
            # calculating left and right eyes using the corner points
            le_x = (faceFeat[0]['left_eye'][3][0]+faceFeat[0]['left_eye'][0][0])/2
            le_y = (faceFeat[0]['left_eye'][3][1]+faceFeat[0]['left_eye'][0][1])/2
            leftEye = [le_x,le_y]

            re_x = (faceFeat[0]['right_eye'][3][0]+faceFeat[0]['right_eye'][0][0])/2
            re_y = (faceFeat[0]['right_eye'][3][1]+faceFeat[0]['right_eye'][0][1])/2
            rightEye = [re_x,re_y]
            
            # calculating the nose by getting the x coordinate of the nose and
            # by getting the y coordinate and subtracting 10 pixels so we get
            # 10 pixels above the nose tip
            n_x = faceFeat[0]['nose_tip'][2][0]
            n_y = faceFeat[0]['nose_tip'][2][1]-10
            nose = [n_x,n_y]
            
            # calculating the mouth by averging the first point of the top and
            # the first point of the bottom lips
            m_x = (faceFeat[0]['top_lip'][0][0]+faceFeat[0]['bottom_lip'][0][0])/2
            m_y = (faceFeat[0]['top_lip'][0][1]+faceFeat[0]['bottom_lip'][0][1])/2
            mouth = [m_x,m_y]

            features.append(leftEye)
            features.append(rightEye)
            features.append(nose)
            features.append(mouth)     
            facefeatures.append(features)
    return facefeatures

In [16]:
# Parameter:  test_data is the features of each image
    
# We go throught mat files for males and females 
# We append the features (left eye, right eye, nose and mouth points )to mf_data list
# we identify each feature for males and females
# we train the data using Quadratic Discriminant Analysis classifier 
# to predict whether the face is male or female
# we return a list with prediction (males, females or none)


def get_pred(test_data):
    mf_data = []
    mf_gender = []

    male = glob.glob('.\\train_data\\male\\*.mat')
    female = glob.glob('.\\train_data\\female\\*.mat')
    
    # Extracting face features from the mat files and adding them to mf_data list 
    # and adding the corresponding gender to mf_gender list
    for i in range(100,200):
        mat_m = scipy.io.loadmat(male[i])
        mat_f = scipy.io.loadmat(female[i])
 
        mf_data.append([[mat_m['x'][0][0],mat_m['y'][0][0]],
                          [mat_m['x'][1][0],mat_m['y'][1][0]],
                          [mat_m['x'][2][0],mat_m['y'][2][0]],
                          [mat_m['x'][3][0],mat_m['y'][3][0]]])

        mf_gender.append('male')

        mf_data.append([[mat_f['x'][0][0],mat_f['y'][0][0]],
                            [mat_f['x'][1][0],mat_f['y'][1][0]],
                            [mat_f['x'][2][0],mat_f['y'][2][0]],
                            [mat_f['x'][3][0],mat_f['y'][3][0]]])

        mf_gender.append('female')

    rfc_prediction = ['None']
    
    X=np.array(mf_data)
    X=X.reshape(X.shape[0], -1)
    Y=np.array(mf_gender)
    if (len(test_data) > 0):
        T=np.array(test_data)
        T=T.reshape(T.shape[0], -1)

        # QuadraticDiscriminantAnalysis classifier
        rfc_clf = QuadraticDiscriminantAnalysis()
        rfc_clf.fit(X,Y)
        rfc_prediction = rfc_clf.predict(T)

    return rfc_prediction

In [17]:
# Parameters: 
# filename: path name for each clip
# logoPic: logo pic for each clip

# we use all the pathnames for each clip to go through each frame in the clip
# we find the face features for each frame using get_feat function 
# we use it as test data in the training model 
# we predict the test data whether each face is male or female using get_pred function
# Then we match logoPic with the logo in the image if it exists we draw a rectangle around it
# Using face_recognition library we detect each face in the image using face_locations function
# we detect each face in the picture and draw a rectangle around it and check if it is male we draw
# a blue rectangle, if it is female we draw a red rectangle.

def detectGenderLogo(filename, logoPic):
    # we find all the pathnames for each clip
    clip = glob.glob(filename)
    logo = cv2.imread(logoPic,0)
    
    for image in clip:
        test_data = get_feat(image)
        features = get_pred(test_data)

        img = cv2.imread(image)
        img_gray = cv2.imread(image,0)

        width,length = logo.shape[::-1]
        thresh = 0.8
        
        # match the logo in the image
        match = cv2.matchTemplate(img_gray,logo, cv2.TM_CCOEFF_NORMED)
        
        # find the location point 
        location = np.where( match >= thresh)
        for point in zip(*location[::-1]):
            cv2.rectangle(img, point, (point[0] + width, point[1]+ length), (0,255,0),3)
        
        # Using face_location library we detect each face in the image
        faces = face_recognition.face_locations(img)

        for i in range(min(len(faces), len(features))):

            width = faces[i][1] - faces[i][3]
            height = faces[i][2] - faces[i][0]
            x = faces[i][3]
            y = faces[i][0]
            
            # We check if the face is male or female and apply a different color for the box
            # and gender text
            if (features[i] == 'male'):
                cv2.rectangle(img, (x,y),(x+width, y+height),(255,0,0),2)
                cv2.putText(img,features[i],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)
            elif (features[i] == 'female'):
                cv2.rectangle(img, (x,y),(x+width, y+height),(0,0,255),2)
                cv2.putText(img,features[i],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),2)
            else:
                cv2.rectangle(img, (x,y),(x+width, y+height),(0,0,0),2)
                cv2.putText(img,features[i],(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
#         io.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#         show()
        cv2.imshow('detected', img)
        k = cv2.waitKey(30)
        if k ==27:
            break


In [20]:
# calling the detectGenderLogo for each clip
detectGenderLogo('./clip_1/*.jpg','./cnbc.jpg')
detectGenderLogo('./clip_2/*.jpg','./clever_news.jpg')
detectGenderLogo('./clip_3/*.jpg','./marvel.jpg')